In [24]:
import pandas as pd
import numpy as np

# Create a function to build the DCF model
# KEY FINANCIAL DATA FROM SHAREHOLDER LETTER
# --------------------------------------------
# Historical revenue & projections
revenue_fy24 = 2700.5  # From shareholder letter (FY24 revenue)
revenue_fy25_projected = 2462.5  # Midpoint of FY25 guidance ($2,455.0M to $2,470.0M)

# Current financial metrics
gross_margin_fy25 = 0.50  # FY25 guidance of 50.0%
adjusted_ebitda_fy25 = 340  # Midpoint of FY25 guidance ($330M to $350M)
free_cash_flow_fy25 = 250  # "in the vicinity of $250 million" for FY25

# Balance sheet items
total_debt = 1499.2  # As of March 31, 2025
cash = 914.3  # As of March 31, 2025

# Share count
shares_outstanding = 394.01  # Weighted-average shares as of Q3 FY25

# Create DCF model dataframe
years = ['FY25', 'FY26', 'FY27', 'FY28', 'FY29', 'FY30', 'Terminal']
dcf = pd.DataFrame(index=years)

# ASSUMPTIONS
# -----------

# Revenue growth assumptions
# The company has been reporting declining revenues but showing operational improvements
# and strategic initiatives that suggest potential stabilization and eventual return to growth
revenue_growth = {
    'FY25': revenue_fy25_projected / revenue_fy24 - 1,  # ~-8.8% (from guidance)
    'FY26': -0.02,  # Slight decline as company continues to stabilize
    'FY27': 0.01,   # Return to slight growth
    'FY28': 0.03,   # Continued modest growth
    'FY29': 0.04,   # Accelerating growth
    'FY30': 0.05,   # Continued growth
    'Terminal': 0.03  # Long-term sustainable growth rate
}

# EBITDA margin assumptions
# Current adjusted EBITDA margin is improving (13.8% for FY25 based on guidance)
# Company is focusing on cost efficiency and profitable growth
ebitda_margin = {
    'FY25': adjusted_ebitda_fy25 / revenue_fy25_projected,  # ~13.8%
    'FY26': 0.15,  # Continued margin improvement
    'FY27': 0.16,  # Further margin improvement
    'FY28': 0.17,  # Continued improvement from operational excellence
    'FY29': 0.18,  # Approaching mature margin profile
    'FY30': 0.18,  # Stabilized margin
    'Terminal': 0.18  # Long-term sustainable margin
}

# D&A as % of revenue (based on recent quarters)
# Q3 FY25 D&A was $21.2M on $624.0M revenue = ~3.4%
da_percent = 0.034

# Capital expenditures as % of revenue
# Recent capex has been minimal ($2.1M in Q3 FY25 on $624.0M revenue = ~0.3%)
# Using a slightly higher figure for long-term sustainability
capex_percent = 0.015

# Net working capital (NWC) as % of revenue change
nwc_percent = 0.10

# Tax rate assumption
tax_rate = 0.25  # Standard corporate tax rate assumption

# WACC calculation
# Risk-free rate (10-year Treasury yield)
risk_free_rate = 0.04
# Market risk premium
market_risk_premium = 0.06
# Peloton beta (assuming 1.5 given high volatility in the stock)
beta = 1.5
# Cost of equity
cost_of_equity = risk_free_rate + beta * market_risk_premium
# Cost of debt (based on existing debt)
cost_of_debt = 0.085  # Estimated from interest expense in shareholder letter
# After-tax cost of debt
after_tax_cost_of_debt = cost_of_debt * (1 - tax_rate)
# Equity weight (based on market cap vs enterprise value)
# Assuming market cap is around $2.5B based on recent trading
market_cap = 2500
V = market_cap + total_debt
equity_weight = market_cap / V
debt_weight = 1 - equity_weight
# WACC calculation
wacc = equity_weight * cost_of_equity + debt_weight * after_tax_cost_of_debt

# BUILDING THE DCF MODEL
# ---------------------

# Revenue projection
dcf.loc['FY25', 'Revenue'] = revenue_fy25_projected
for i in range(1, len(years)):
    dcf.loc[years[i], 'Revenue'] = dcf.loc[years[i-1], 'Revenue'] * (1 + revenue_growth[years[i]])

# EBITDA calculation
for year in years:
    dcf.loc[year, 'EBITDA Margin'] = ebitda_margin[year]
    dcf.loc[year, 'EBITDA'] = dcf.loc[year, 'Revenue'] * dcf.loc[year, 'EBITDA Margin']

# D&A calculation
dcf['D&A'] = dcf['Revenue'] * da_percent

# EBIT calculation
dcf['EBIT'] = dcf['EBITDA'] - dcf['D&A']

# Taxes calculation
dcf['Taxes'] = dcf['EBIT'] * tax_rate

# NOPAT (Net Operating Profit After Tax)
dcf['NOPAT'] = dcf['EBIT'] - dcf['Taxes']

# Add back D&A
dcf['Add D&A'] = dcf['D&A']

# Capex calculation
dcf['Capex'] = dcf['Revenue'] * capex_percent

# Change in NWC calculation
dcf['Change in Revenue'] = dcf['Revenue'].diff()
dcf.loc['FY25', 'Change in Revenue'] = 0  # No change for first year in model
dcf['Change in NWC'] = dcf['Change in Revenue'] * nwc_percent

# Free Cash Flow calculation
dcf['FCF'] = dcf['NOPAT'] + dcf['Add D&A'] - dcf['Capex'] - dcf['Change in NWC']

# Terminal value calculation
terminal_fcf = dcf.loc['Terminal', 'FCF']
terminal_value = terminal_fcf * (1 + revenue_growth['Terminal']) / (wacc - revenue_growth['Terminal'])

# Discount factors
discount_factors = [(1 + wacc) ** -i for i in range(1, len(years))]
for i, year in enumerate(years[:-1]):  # Exclude terminal year
    dcf.loc[year, 'Discount Factor'] = discount_factors[i]

# Present value of FCFs
for year in years[:-1]:  # Exclude terminal year
    dcf.loc[year, 'PV of FCF'] = dcf.loc[year, 'FCF'] * dcf.loc[year, 'Discount Factor']

# Present value of terminal value
pv_terminal_value = terminal_value * discount_factors[-1]  # Use last explicit period discount factor

# Enterprise value calculation
enterprise_value_dcf = dcf['PV of FCF'].sum() + pv_terminal_value

# Equity value calculation
equity_value = enterprise_value_dcf - total_debt + cash

# Share price calculation
share_price = equity_value / shares_outstanding

# Create summary table
summary = pd.DataFrame({
    'Metric': [
        'DCF Enterprise Value ($M)',
        'Less: Total Debt ($M)',
        'Add: Cash ($M)',
        'Equity Value ($M)',
        'Shares Outstanding (M)',
        'Share Price ($)',
        '',
        'WACC Calculation',
        'Risk-free Rate',
        'Market Risk Premium',
        'Beta',
        'Cost of Equity',
        'Cost of Debt (Pre-tax)',
        'Cost of Debt (After-tax)',
        'Equity Weight',
        'Debt Weight',
        'WACC',
        '',
        'Key Assumptions',
        'Terminal Growth Rate',
        'Terminal EBITDA Margin',
        'Tax Rate'
    ],
    'Value': [
        f"{enterprise_value_dcf:.1f}",
        f"{total_debt:.1f}",
        f"{cash:.1f}",
        f"{equity_value:.1f}",
        f"{shares_outstanding:.1f}",
        f"${share_price:.2f}",
        '',
        '',
        f"{risk_free_rate:.1%}",
        f"{market_risk_premium:.1%}",
        f"{beta:.2f}",
        f"{cost_of_equity:.1%}",
        f"{cost_of_debt:.1%}",
        f"{after_tax_cost_of_debt:.1%}",
        f"{equity_weight:.1%}",
        f"{debt_weight:.1%}",
        f"{wacc:.1%}",
        '',
        '',
        f"{revenue_growth['Terminal']:.1%}",
        f"{ebitda_margin['Terminal']:.1%}",
        f"{tax_rate:.1%}"
    ]
})

    # Return the model, enterprise value, equity value, and share price
result =  {
        'dcf_model': dcf,
        'summary': summary,
        'enterprise_value': enterprise_value_dcf,
        'equity_value': equity_value,
        'share_price': share_price,
        'wacc': wacc,
        'assumptions': {
            'revenue_growth': revenue_growth,
            'ebitda_margin': ebitda_margin,
            'capex_percent': capex_percent,
            'nwc_percent': nwc_percent,
            'tax_rate': tax_rate,
            'terminal_growth': revenue_growth['Terminal']
        }
    }

In [25]:
# Display the DCF model
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print("====== PELOTON DCF VALUATION MODEL ======")
print("\nDCF Model:")
print(result['dcf_model'].round(1))

print("\nValuation Summary:")
print(result['summary'])

print("\nKey Results:")
print(f"Enterprise Value: ${result['enterprise_value']:.1f} million")
print(f"Equity Value: ${result['equity_value']:.1f} million")
print(f"Share Price: ${result['share_price']:.2f}")
print(f"WACC: {result['wacc']:.1%}")

====== PELOTON DCF VALUATION MODEL ======

DCF Model:
          Revenue  EBITDA Margin  EBITDA   D&A   EBIT  Taxes  NOPAT  Add D&A  Capex  Change in Revenue  Change in NWC    FCF  Discount Factor  PV of FCF
FY25       2462.5            0.1   340.0  83.7  256.3   64.1  192.2     83.7   36.9                0.0            0.0  239.0              0.9      216.3
FY26       2413.2            0.2   362.0  82.1  279.9   70.0  210.0     82.1   36.2              -49.2           -4.9  260.7              0.8      213.5
FY27       2437.4            0.2   390.0  82.9  307.1   76.8  230.3     82.9   36.6               24.1            2.4  274.2              0.7      203.2
FY28       2510.5            0.2   426.8  85.4  341.4   85.4  256.1     85.4   37.7               73.1            7.3  296.5              0.7      198.7
FY29       2610.9            0.2   470.0  88.8  381.2   95.3  285.9     88.8   39.2              100.4           10.0  325.5              0.6      197.4
FY30       2741.5           